In [32]:
import pandas as pd

In [33]:
df = pd.read_csv('battle_results.csv')

df['p1win'] = df['result'] == 1
df['p2win'] = df['result'] == 2
df['tie'] = df['result'] == 0

df['p1name'] = df['p1name'].str.title()
df['p2name'] = df['p2name'].str.title()

df = df.drop(columns=['Unnamed: 0'])

df.head(5)

,player1,player2,result,p1name,p2name,p1win,p2win,tie
0,0,0,2,Scyther,Scyther,False,True,False
1,0,1,2,Scyther,Pinsir,False,True,False
2,0,2,2,Scyther,Moltres,False,True,False
3,0,3,2,Scyther,Moltres Ex,False,True,False
4,0,4,2,Scyther,Heatmor,False,True,False


In [34]:
evo_names = df.groupby('player1')['p1name'].first().reset_index().rename(columns={'player1': 'id', 'p1name': 'name'})

evo_names.head(5)

,id,name
0,0,Scyther
1,1,Pinsir
2,2,Moltres
3,3,Moltres Ex
4,4,Heatmor


In [35]:
def get_losses(id:int) -> set[int]:
    p1 = df[(df['player1'] == id) & ~(df['p1win'])]
    p2 = df[(df['player2'] == id) & ~(df['p2win'])]
    return set([int(item) for item in set(p1['player2'].unique()) | set(p2['player1'].unique())])

In [36]:
loss74 = get_losses(74)
loss75 = get_losses(75)

eggLosses = loss74 & loss75

named_evo_list = lambda s: [evo_names.loc[item, 'name'].title() for item in s]

print(f"Common losses:           {named_evo_list(eggLosses)}")
print(f"Egg 74 unique losses:    {named_evo_list(loss74 - eggLosses)}")
print(f"Egg 75 unique losses:    {named_evo_list(loss75 - eggLosses)}")

Common losses:           ['Floatzel', 'Exeggutor Ex', 'Exeggutor Ex', 'Rampardos', 'Infernape Ex']
Egg 74 unique losses:    ['Exeggutor', 'Glaceon', 'Bastidon']
Egg 75 unique losses:    ['Seaking', 'Moltres Ex', 'Exeggutor', 'Aerodactyl Ex', 'Rapidash', 'Rapidash', 'Rapidash']


In [37]:
max_battles = int((df['player1'].max() + 1) * 2) - 1
max_battles

615

In [38]:
(df['result'].value_counts() / df['result'].count() * 100).round(2)

result
2    55.4
1    44.5
0     0.1
Name: count, dtype: float64

In [39]:
player_wins = df.groupby('player1')[['p1win', 'tie']].sum().rename(columns={'p1win': 'player_win', 'tie': "player_tie"})
opponent_wins = df[df['player1'] != df['player2']].groupby('player2')[['p2win', 'tie']].sum().rename(columns={'p2win': 'opponent_win', 'tie': "opponent_tie"})

wins = player_wins.merge(opponent_wins, left_index=True, right_index=True).reset_index().rename(columns={'player1': 'id'})
wins['wins'] = wins['player_win'] + wins['opponent_win']

wins = wins.set_index('id').merge(evo_names.set_index('id'), right_index=True, left_index=True).reset_index()

wins['win_perc'] = (wins['wins'] / max_battles * 100).round(2)

wins.head(5)

,id,player_win,player_tie,opponent_win,opponent_tie,wins,name,win_perc
0,0,45,0,85,0,130,Scyther,21.14
1,1,129,0,216,0,345,Pinsir,56.10
2,2,138,0,229,0,367,Moltres,59.67
3,3,205,0,256,0,461,Moltres Ex,74.96
4,4,61,0,101,0,162,Heatmor,26.34


In [40]:
wins[['id', 'wins', 'win_perc', 'name']].sort_values('wins', axis=0, ascending=False).head(20)

,id,wins,win_perc,name
74,74,607,98.70,Exeggutor Ex
75,75,604,98.21,Exeggutor Ex
300,300,593,96.42,Rampardos
289,289,592,96.26,Magnezone
290,290,589,95.77,Magnezone
286,286,582,94.63,Infernape Ex
176,176,579,94.15,Aerodactyl Ex
99,99,572,93.01,Starmie Ex
73,73,560,91.06,Exeggutor
243,243,560,91.06,Blastoise Ex


In [41]:
list(set(named_evo_list(get_losses(195))))

['Machamp',
 'Mewtwo Ex',
 'Bisharp',
 'Wormadam',
 'Golem',
 'Electivire',
 'Yanmega Ex',
 'Glaceon',
 'Vespiqueen',
 'Palkia Ex',
 'Weavile Ex',
 'Florges',
 'Exeggutor Ex',
 'Kabutops',
 'Cresselia',
 'Omastar',
 'Leafeon',
 'Dusknoir',
 'Lapras Ex',
 'Primape',
 'Weezing',
 'Roserade',
 'Alakazam',
 'Bellossom',
 'Infernape Ex',
 'Beedrill',
 'Pidgeot Ex',
 'Darkrai Ex',
 'Magnezone',
 'Lumineon',
 'Cresselia Ex',
 'Pachirisu Ex',
 'Regice',
 'Arbok',
 'Scolipede',
 'Vaporeon',
 'Gengar',
 'Drednaw',
 'Gengar Ex',
 'Mamoswine',
 'Blastoise Ex',
 'Togekiss',
 'Regirock',
 'Dialga Ex',
 'Heliolisk',
 'Aerodactyl Ex',
 'Electrode',
 'Eelektross',
 'Exeggutor',
 'Marowak Ex',
 'Zebstrika',
 'Wigglytuff',
 'Machamp Ex',
 'Gallade Ex',
 'Himontop',
 'Floatzel',
 'Bastidon',
 'Pidgeot',
 'Luxray',
 'Ambipom',
 'Articuno Ex',
 'Rampardos',
 'Starmie Ex']

In [42]:
df[(df['player1']==195) & (df['p1win']) & (df['p2name'].str.contains(" ex", case=False)) | ((df['player2']==195) & (df['p2win'])  & (df['p1name'].str.contains(" ex", case=False)))]

,player1,player2,result,p1name,p2name,p1win,p2win,tie
1119,3,195,2,Moltres Ex,Floatzel,False,True,False
2351,7,195,2,Articuno Ex,Floatzel,False,True,False
3275,10,195,2,Pikachu Ex,Floatzel,False,True,False
3891,12,195,2,Zapdos Ex,Floatzel,False,True,False
5431,17,195,2,Mewtwo Ex,Floatzel,False,True,False
9127,29,195,2,Celebi Ex,Floatzel,False,True,False
10359,33,195,2,Mew Ex,Floatzel,False,True,False
14671,47,195,2,Palkia Ex,Floatzel,False,True,False
15287,49,195,2,Pachirisu Ex,Floatzel,False,True,False
18675,60,195,2,Darkrai Ex,Floatzel,False,True,False


In [43]:
df[((df['player1'] == 75) & ~(df['p1win'])) | ((df['player2'] == 75) & ~(df['p2win']))]

,player1,player2,result,p1name,p2name,p1win,p2win,tie
22251,72,75,1,Exeggutor,Exeggutor Ex,True,False,False
23103,75,3,2,Exeggutor Ex,Moltres Ex,False,True,False
23174,75,74,2,Exeggutor Ex,Exeggutor Ex,False,True,False
23175,75,75,1,Exeggutor Ex,Exeggutor Ex,True,False,False
23254,75,154,2,Exeggutor Ex,Rapidash,False,True,False
23255,75,155,2,Exeggutor Ex,Rapidash,False,True,False
25639,83,75,1,Rapidash,Exeggutor Ex,True,False,False
29951,97,75,1,Seaking,Exeggutor Ex,True,False,False
54283,176,75,1,Aerodactyl Ex,Exeggutor Ex,True,False,False
60135,195,75,1,Floatzel,Exeggutor Ex,True,False,False


In [44]:
wins[['id', 'player_win', 'name']].sort_values('player_win', axis=0, ascending=False).head(20)

,id,player_win,name
300,300,308,Rampardos
74,74,305,Exeggutor Ex
75,75,304,Exeggutor Ex
289,289,302,Magnezone
290,290,302,Magnezone
176,176,300,Aerodactyl Ex
286,286,289,Infernape Ex
99,99,288,Starmie Ex
234,234,284,Beedrill
261,261,282,Kabutops


In [45]:
df[df['tie']].groupby('p1name')['result'].count().reset_index().sort_values('result', ascending=False)

,p1name,result
7,Lumineon,18
5,Hitmonlee,12
1,Chatot,10
8,Manaphy,10
9,Mespirit,10
3,Ditto,8
10,Mew Ex,8
4,Fan Rotom,7
0,Aerodactyl,4
2,Cinccino,3


In [46]:
wins[wins['wins'] <= 5][['id', 'name', 'wins']].sort_values('wins', ascending=False)

,id,name,wins
165,165,Lumineon,2
166,166,Lumineon,1
19,19,Hitmonlee,0
39,39,Chatot,0
48,48,Manaphy,0
52,52,Mespirit,0
